In [2]:
cd ..

/Users/alexwang/Downloads/Projects/quant_strategy_repo


# read data

In [31]:
from utilities.util import load_config, today, save_file, log, drop_columns
import pandas as pd 

def load_data(filepath):
    log(f"Loading data from {filepath}")
    return pd.read_csv(filepath, parse_dates=["Date"], index_col=["Date"])

config = load_config()
filepath = "data/processed_data/data_clean.csv"
cutoff_date = config["cutoff_date"]

data = load_data(filepath)

print(data.Ticker.nunique())
data

2024-03-17 19:07:22,706 - INFO - Loading data from data/processed_data/data_clean.csv


4085


,Ticker,Adj Close,Volume,EPS Estimate,Reported EPS,Surprise (%),period_max_price,period_min_price
Date,,,,,,,,
2015-01-02,A,38.134743,1529200.0,NaN,NaN,NaN,NaN,NaN
2015-01-05,A,37.420185,2041800.0,NaN,NaN,NaN,NaN,NaN
2015-01-06,A,36.837265,2080600.0,NaN,NaN,NaN,NaN,NaN
2015-01-07,A,37.326168,3359700.0,NaN,NaN,NaN,NaN,NaN
2015-01-08,A,38.445004,2116300.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
2024-03-08,ZYXI,12.860000,213800.0,NaN,NaN,NaN,NaN,NaN
2024-03-11,ZYXI,12.690000,151500.0,NaN,NaN,NaN,NaN,NaN
2024-03-12,ZYXI,12.600000,151100.0,NaN,NaN,NaN,NaN,NaN


# check 0 vol tickers

In [32]:
# data.Ticker.value_counts().sort_values(ascending=False)
# data['Volume'].describe()
df_zero_vol = data.query('Volume == 0')['Ticker'].value_counts()
tickers_zero_vol = df_zero_vol[df_zero_vol > 0].index


data = data.query('~Ticker.isin(@tickers_zero_vol)')
display(data)
len(tickers_zero_vol)

,Ticker,Adj Close,Volume,EPS Estimate,Reported EPS,Surprise (%),period_max_price,period_min_price
Date,,,,,,,,
2015-01-02,A,38.134743,1529200.0,NaN,NaN,NaN,NaN,NaN
2015-01-05,A,37.420185,2041800.0,NaN,NaN,NaN,NaN,NaN
2015-01-06,A,36.837265,2080600.0,NaN,NaN,NaN,NaN,NaN
2015-01-07,A,37.326168,3359700.0,NaN,NaN,NaN,NaN,NaN
2015-01-08,A,38.445004,2116300.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
2024-03-08,ZYME,11.440000,915600.0,NaN,NaN,NaN,NaN,NaN
2024-03-11,ZYME,11.050000,407400.0,NaN,NaN,NaN,NaN,NaN
2024-03-12,ZYME,10.810000,760700.0,NaN,NaN,NaN,NaN,NaN


585

In [33]:
data.Ticker.nunique()

3500

# filter by EPS estimate

In [41]:
data.dropna(subset=['EPS Estimate'], inplace=True)
df_eps_vount = data.Ticker.value_counts().sort_values(ascending=False)
df_eps_vount.describe()

/var/folders/b_/2ptnl6bd2xbdd5tqmj651vdm0000gn/T/ipykernel_74765/252779906.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.dropna(subset=['EPS Estimate'], inplace=True)


count    3088.000000
mean       29.745466
std        11.238440
min         1.000000
25%        22.000000
50%        37.000000
75%        37.000000
max        65.000000
Name: count, dtype: float64

In [45]:
tickers_few_eps_estimates = df_eps_vount[df_eps_vount < 10]
print(len(tickers_few_eps_estimates))
data = data.query('~Ticker.isin(@tickers_few_eps_estimates.index)')
print(data.Ticker.nunique())
data

210
2878


,Ticker,Adj Close,Volume,EPS Estimate,Reported EPS,Surprise (%),period_max_price,period_min_price
Date,,,,,,,,
2015-02-17,A,38.097126,2875800.0,0.41,0.41,-0.52,41.046803,38.200565
2015-05-18,A,40.179691,1966100.0,0.39,0.38,-1.46,40.160835,36.434608
2015-08-17,A,36.680275,3302100.0,0.41,0.44,8.20,36.869247,31.622189
2015-11-16,A,35.374790,2490200.0,0.47,0.50,5.82,40.141327,33.067478
2016-02-16,A,35.233955,2903600.0,0.43,0.46,6.36,40.720184,35.319477
...,...,...,...,...,...,...,...,...
2023-03-07,ZYME,7.960000,543800.0,4.84,4.65,-3.93,10.220000,7.930000
2023-05-08,ZYME,8.760000,424900.0,-0.54,-0.36,32.91,9.980000,7.260000
2023-08-10,ZYME,7.760000,676200.0,-0.47,-0.76,-61.55,7.970000,6.180000


In [48]:
final_selected_tickers = data[['Ticker']].drop_duplicates()
save_file(final_selected_tickers, "data/market_data/selected_tickers.csv", index=False)
final_selected_tickers

,Ticker
Date,
2015-02-17,A
2016-12-01,AA
2018-05-15,AADI
2015-01-27,AAL
2015-02-06,AAN
...,...
2015-02-11,ZTS
2015-03-12,ZUMZ
2018-05-31,ZUO
